In [1]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt
import torch.optim as optim
torch.set_default_tensor_type(torch.DoubleTensor) # this ensure double precision, single precision is not accurate

In [28]:
#GD for 3by3 system 
print("Plain GD: number of iterations needed for 3 by 3 system")
for eps in [0.1,0.01,0.001,1e-4,1e-5,1e-9, 0.]: 
    A3 = torch.tensor([[1+eps,-1,0],[-1,2+eps,-1],[0,-1,1+eps]])
    x = torch.zeros(3)
    x = x.view(3,1)
    b = torch.tensor([[-1.],[-1.],[2.]]) # must be in kernel is eps = 0
    x.data = torch.tensor([[1.0],[2.],[3.0]])
    tol = 1e-8 # tolerance for residual norm 
    residual_norm = torch.norm(torch.matmul(A3,x) -b,2)
    iters = 0 
    while residual_norm > tol: 
        gd = torch.matmul(A3,x) - b 
        x = x - 0.5*gd 
        residual_norm = torch.norm(gd,2)
        iters += 1 
        if iters > 1000000: 
            break
        assert torch.isnan(residual_norm)!=True, "norm is nan, reset learning rate" #somehow nan>tol returns false
    if iters > 1000000: 
        print("eps = "+str(eps)+": over 1,000,000")
    else:
        print("eps = "+str(eps)+": ", iters)

Plain GD: number of iterations needed for 3 by 3 system
eps = 0.1:  340
eps = 0.01:  3006
eps = 0.001:  25506
eps = 0.0001:  209052
eps = 1e-05: over 1,000,000
eps = 1e-09:  29
eps = 0.0:  29


In [37]:
#GD for 4by4 system, plain GD
print("Plain GD: 4by4 system")
P = torch.tensor([[1.,0.,0.,1.],[0.,1.,0.,1.],[0.,0.,1.,1.]])
for eps in [0.1,0.01,0.001,1e-4,1e-5,1e-9,0.]: 
    A3 = torch.tensor([[1+eps,-1,0],[-1,2+eps,-1],[0,-1,1+eps]])
    A4 = torch.tensor([[1+eps,-1.0,0,eps],[-1,2+eps,-1,eps],[0,-1,1+eps,eps],[eps,eps,eps,3*eps]])
    x = torch.zeros(4)
    x = x.view(4,1)
    b = torch.tensor([[-1.],[-1.],[2.],[0.]]) #
    tol = 1e-6 
#     residual_norm = torch.norm(torch.matmul(A4,x) -b,2)
    residual_norm = torch.norm(A3@(P@x)-P@b,2)
    iters = 0 
    while residual_norm > tol: 
        gd = torch.matmul(A4,x) - b 
        x.data = x.data - 0.5*gd 
#         residual_norm = torch.norm(torch.matmul(A4,x) - b,2)
        residual_norm = torch.norm(A3@(P@x)-P@b,2)
        iters += 1 
        if iters > 100000: 
            break
    assert torch.isnan(residual_norm)!=True, "norm is nan, reset learning rate"
    print("eps = "+str(eps)+": ", iters)
#     print(torch.matmul(P,x))
print()


#GD for 4by4 system, Jacobi preconditioner
print("Scaled GD: 4by4 system")
P = torch.tensor([[1.,0.,0.,1.],[0.,1.,0.,1.],[0.,0.,1.,1.]])
for eps in [0.1,0.01,0.001,1e-4,1e-5,1e-9]: 
    A3 = torch.tensor([[1+eps,-1,0],[-1,2+eps,-1],[0,-1,1+eps]])
    A4 = torch.tensor([[1+eps,-1.0,0,eps],[-1,2+eps,-1,eps],[0,-1,1+eps,eps],[eps,eps,eps,3*eps]])
    D = torch.diag(torch.diag(A4))
    x = torch.zeros(4)
    x = x.view(4,1)
    b = torch.tensor([[-1.],[-1.],[2.],[0.]]) #
    tol = 1e-6
#     residual_norm = torch.norm(torch.matmul(A4,x) -b,2)
    residual_norm = torch.norm(A3@(P@x)-P@b,2)
    iters = 0 
    while residual_norm > tol: 
        gd = torch.matmul(A4,x) - b 
        x.data = x.data - 0.7*torch.matmul(torch.linalg.inv(D),gd) # A larger step size is allowed
#         residual_norm = torch.norm(torch.matmul(A4,x) - b,2)
        residual_norm = torch.norm(A3@(P@x)-P@b,2)
        iters += 1 
        if iters > 100000: 
            break
    assert torch.isnan(residual_norm)!=True, "norm is nan, reset learning rate"
    print("eps = "+str(eps)+": ", iters)
#     print(torch.matmul(P,x))
    

Plain GD: 4by4 system
eps = 0.1:  24
eps = 0.01:  22
eps = 0.001:  22
eps = 0.0001:  22
eps = 1e-05:  22
eps = 1e-09:  22
eps = 0.0:  22

Scaled GD: 4by4 system
eps = 0.1:  14
eps = 0.01:  16
eps = 0.001:  16
eps = 0.0001:  16
eps = 1e-05:  16
eps = 1e-09:  16
